In [1]:
!curl -o 'weather.json' 'https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2010-01-01&end_date=2023-08-31&hourly=temperature_2m,relativehumidity_2m,rain,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m,winddirection_10m,winddirection_100m'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8605k    0 8605k    0     0  1272k      0 --:--:--  0:00:06 --:--:-- 1351kk    0     0  1272k      0 --:--:--  0:00:04 --:--:-- 1273k


In [2]:
import json 
import pandas as pd

f = open('weather.json')
data = json.load(f)
f.close()


df = pd.DataFrame.from_dict(data["hourly"])

In [3]:
import sklearn.model_selection
from lakefs_spec import LakeFSFileSystem


fs = LakeFSFileSystem()

In [4]:
from lakefs_client.client import LakeFSClient
from lakefs_spec.client_helpers import commit
from lakefs_spec.hooks import FSEvent, HookContext

#Define the commit hook
def commit_on_put(client: LakeFSClient, ctx:HookContext) -> None:
    commit_message = f"Add file {ctx.resource}"
    print(f"Attempting Commit: {commit_message}")
    commit(client, repository=ctx.repository, branch = ctx.ref, message=commit_message)
    

#Register the commit hook to be executed after a PUT_FILE event
fs.register_hook(FSEvent.PUT_FILE, commit_on_put)

In [5]:

train, test = sklearn.model_selection.train_test_split(df)

df2 = pd.concat([train,test])

train, test = sklearn.model_selection.train_test_split(df2)

test.to_csv('tts_test.csv')
train.to_csv(f'lakefs://quickstart/weird/tts_test.csv')
train.to_csv(f'tts_train.csv')
train.to_csv(f'lakefs://quickstart/weird/tts_train.csv')


tts_weird_from_lakefs = pd.read_csv(f'lakefs://quickstart/weird/tts_train.csv', index_col=0)
tts_weird_from_local =  pd.read_csv(f'tts_train.csv', index_col=0)

test_weird_lakefs = pd.read_csv(f'lakefs://quickstart/weird/tts_test.csv', index_col=0)
test_weird_local = pd.read_csv(f'tts_test.csv', index_col=0)


Calling open() in write mode results in unbuffered file uploads, because the lakeFS Python client does not support multipart uploads. Note that uploading large files unbuffered can have performance implications.
Attempting Commit: Add file tts_test.csv
Calling open() in write mode results in unbuffered file uploads, because the lakeFS Python client does not support multipart uploads. Note that uploading large files unbuffered can have performance implications.
Attempting Commit: Add file tts_train.csv


In [6]:
tts_weird_from_lakefs

,2020-08-31T21:00,15.5,84,0.0,1017.5,1012.9,7,5,4,0,8.0,18.4,10,17
93501,,,,,,,,,,,,,,
22653,2012-08-01T21:00,19.3,67,0.0,1015.0,1010.5,0,0,0,0,11.2,25.8,94,102
58315,2016-08-26T19:00,26.3,41,0.0,1016.8,1012.4,0,0,0,0,11.3,26.3,153,163
94749,2020-10-22T21:00,15.6,75,0.0,1012.2,1007.7,43,7,28,66,16.0,30.9,226,229
114634,2023-01-29T10:00,1.2,88,0.0,1024.8,1020.0,100,99,67,96,15.6,24.5,247,250
67635,2017-09-19T03:00,7.0,95,0.0,1015.6,1010.9,2,2,0,0,6.6,16.1,221,241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119595,2023-08-24T03:00,15.5,90,0.0,1018.6,1014.0,59,0,99,0,5.8,14.6,202,220
77954,2018-11-23T02:00,1.3,80,0.0,1024.6,1019.8,96,97,15,0,9.7,17.6,92,101
90597,2020-05-02T21:00,9.0,81,0.0,1009.2,1004.6,34,22,14,19,13.0,24.3,272,278


In [7]:
tts_weird_from_local

,time,temperature_2m,relativehumidity_2m,rain,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m,winddirection_10m,winddirection_100m
3193,2010-05-14T01:00,8.5,84,0.0,1010.4,1005.8,100,98,21,100,8.9,15.5,328,332
20656,2012-05-10T16:00,24.4,56,0.0,1018.8,1014.4,54,1,78,20,9.5,19.4,205,211
105856,2022-01-28T16:00,4.2,65,0.0,1027.3,1022.5,35,12,0,80,18.0,32.3,147,145
17018,2011-12-11T02:00,1.8,84,0.0,1014.3,1009.5,34,38,0,0,17.6,31.6,244,249
34494,2013-12-08T06:00,0.2,99,0.0,1022.5,1017.7,100,100,96,40,8.1,15.1,167,177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119595,2023-08-24T03:00,15.5,90,0.0,1018.6,1014.0,59,0,99,0,5.8,14.6,202,220
77954,2018-11-23T02:00,1.3,80,0.0,1024.6,1019.8,96,97,15,0,9.7,17.6,92,101
90597,2020-05-02T21:00,9.0,81,0.0,1009.2,1004.6,34,22,14,19,13.0,24.3,272,278
48289,2015-07-06T01:00,24.5,68,0.0,1013.8,1009.4,59,0,49,100,11.0,19.9,221,229


In [8]:
test_weird_local

,time,temperature_2m,relativehumidity_2m,rain,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m,winddirection_10m,winddirection_100m
106643,2022-03-02T11:00,4.9,60,0.0,1026.6,1021.8,26,0,2,81,3.3,4.6,347,342
97440,2021-02-12T00:00,-9.1,91,0.0,1032.4,1027.3,69,31,69,0,10.0,14.5,13,30
17876,2012-01-15T20:00,-3.0,86,0.0,1026.2,1021.3,38,0,63,0,6.5,16.2,326,343
45307,2015-03-03T19:00,4.4,75,0.5,1013.6,1008.9,81,2,98,68,15.9,29.0,193,200
33565,2013-10-30T13:00,11.6,64,0.0,1020.8,1016.2,26,17,18,0,19.6,28.6,250,251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39760,2014-07-15T16:00,24.1,47,0.0,1018.1,1013.7,26,0,29,27,11.2,15.9,276,275
52147,2015-12-13T19:00,2.6,86,0.0,1022.4,1017.6,20,11,0,35,11.5,24.5,272,280
31807,2013-08-18T07:00,21.2,72,0.0,1012.3,1007.8,82,0,87,99,12.0,20.3,213,217
58146,2016-08-19T18:00,23.3,46,0.0,1010.9,1006.5,19,0,31,0,6.4,13.2,52,61


In [9]:
test_weird_lakefs

,2020-08-31T21:00,15.5,84,0.0,1017.5,1012.9,7,5,4,0,8.0,18.4,10,17
93501,,,,,,,,,,,,,,
22653,2012-08-01T21:00,19.3,67,0.0,1015.0,1010.5,0,0,0,0,11.2,25.8,94,102
58315,2016-08-26T19:00,26.3,41,0.0,1016.8,1012.4,0,0,0,0,11.3,26.3,153,163
94749,2020-10-22T21:00,15.6,75,0.0,1012.2,1007.7,43,7,28,66,16.0,30.9,226,229
114634,2023-01-29T10:00,1.2,88,0.0,1024.8,1020.0,100,99,67,96,15.6,24.5,247,250
67635,2017-09-19T03:00,7.0,95,0.0,1015.6,1010.9,2,2,0,0,6.6,16.1,221,241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119595,2023-08-24T03:00,15.5,90,0.0,1018.6,1014.0,59,0,99,0,5.8,14.6,202,220
77954,2018-11-23T02:00,1.3,80,0.0,1024.6,1019.8,96,97,15,0,9.7,17.6,92,101
90597,2020-05-02T21:00,9.0,81,0.0,1009.2,1004.6,34,22,14,19,13.0,24.3,272,278
